In [2]:
#imports and defs
import pythia8
from pyjet import cluster, DTYPE_PTEPM
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
plt.style.use("dark_paper")

def show(vec):
    string = (4*"{:.2e} ").format(vec.e(),vec.px(),vec.py(),vec.pz())
    print(string)
def to_array(vec):
    return np.array([vec.e(),vec.px(),vec.py(),vec.pz()])
def prtStable(pid):
    if abs(pid) == 211: return True #pion
    if abs(pid) == 321: return True #K+-
    if abs(pid) == 11: return True #electron
    if abs(pid) == 13: return True #muon?
    if abs(pid) == 2212: return True #proton
    #
    #if abs(pid) == 22: return True #gamma
    #if abs(pid) == 130: return True #K_l
    return False
def heavyFlavor(pid):
    if abs(pid) == 411: return True
    if abs(pid) == 421: return True
    if abs(pid) == 431: return True
    if abs(pid) == 4122: return True
    if abs(pid) == 511: return True
    if abs(pid) == 521: return True
    if abs(pid) == 531: return True
    if abs(pid) == 5122: return True
    return False
def getData(prt):
    data = [prt.index(),prt.name(),prt.id(),prt.status(),prt.mother1(),prt.mother2(),
            prt.daughter1(),prt.daughter2(),prt.e(),prt.px(),prt.py(),prt.pz(),prt.m(),prt.pT(),prt.eta(),prt.phi(),prt.theta()]
    keys = ["N","NAME","ID","STATUS","M1","M2","D1","D2","E", "px", "py", "pz" , "mass","pT","eta","phi","theta"]
    return {key:value for key,value in zip(keys,data)}
def getDataLight(prt):
    data = [prt.index(),prt.name(),prt.id(),prt.status(),
            prt.e(),prt.px(),prt.py(),prt.pz(),prt.m(),prt.pT(),prt.eta(),prt.phi(),prt.theta()]
    keys = ["N","NAME","ID","STATUS","E", "px", "py", "pz" , "mass","pT","eta","phi","theta"]
    return {key:value for key,value in zip(keys,data)}
def getInfo(prt):
    string = "{:^2d} {:^11s} {:^5d} {:^6d} {:<3d} {:>3d}  {:<3d} {:>3d}    ".format(prt.index(),prt.name(),prt.id(),
                                                                       prt.status(),
                                                                       prt.mother1(),prt.mother2(),
                                                                       prt.daughter1(),prt.daughter2())
    string += (5*" {:>8.3f} ").format(prt.e(),prt.px(),prt.py(),prt.pz(),prt.m())
    return string

In [3]:
# Setup Pythia Parameters
# Generator. Shorthand for event.
pythia = pythia8.Pythia()

# Set up incoming beams, for frame with unequal beam energies.
pythia.readString("Beams:frameType = 2")
# BeamA = proton.
pythia.readString("Beams:idA = 2212")
pythia.settings.parm("Beams:eA", 100.)
# BeamB = electron.
pythia.readString("Beams:idB = 11")#11
pythia.settings.parm("Beams:eB", 10.) # used a 100 was good before
# Phase-space cut: minimal Q2 of process.
pythia.settings.parm("PhaseSpace:Q2Min", 10.)
# Set up DIS process within some phase space.
# Neutral current (with gamma/Z interference).
pythia.readString("WeakBosonExchange:ff2ff(t:gmZ) = on")
# Uncomment to allow charged current.
pythia.readString("WeakBosonExchange:ff2ff(t:W) = on")
# Set dipole recoil on. Necessary for DIS + shower.
pythia.readString("SpaceShower:dipoleRecoil = on")
# Allow emissions up to the kinematical limit,
# since rate known to match well to matrix elements everywhere.
pythia.readString("SpaceShower:pTmaxMatch = 2")
# QED radiation off lepton not handled yet by the new procedure.
pythia.readString("PDF:lepton = off")
pythia.readString("TimeShower:QEDshowerByL = off")
# Setting the random seed
pythia.readString("Random:setSeed = on")
pythia.readString("Random:seed = 0")
# Removing FSR and ISR
pythia.readString("PartonLevel:FSR = off")
pythia.readString("PartonLevel:ISR = off")
# # QCD pp
# pythia.readString("HardQCD:all = on");
# pythia.readString("PhaseSpace:pTHatMin = .2");
# Initialize.
print("Pythia generator initialized successfully: ", pythia.init())

Pythia generator initialized successfully:  True


In [4]:
def selection(prt):
    sel = prt.isFinal() and prt.vProd().pAbs()<1000 or abs(prt.id())<=5 or prt.id()==22 #and prtStable(prt.id()) # and prt.isCharged()
    return sel

In [13]:
# keys
cols_to_save = ["EVENT", "N","NAME","ID","STATUS","M1","M2","D1","D2","E", "px", "py", "pz" , "mass","pT","eta","phi","theta"]
#cols_to_save = ["EVENT", 'N', "NAME","ID","STATUS","E", "px", "py", "pz" , "mass","pT","eta","phi","theta"]
convert_dict = {'NAME': 'category', 
                'ID': 'category',
                'mass': 'category',
                'STATUS' : 'int8',
                'EVENT': 'int32',
               } 
for key in ["E", "px", "py", "pz" ,"pT","eta","phi","theta"]:
    convert_dict[key] = 'float32'
for key in ["N", "M1","M2","D1","D2"]:
    convert_dict[key] = 'uint8'

In [14]:
#%%capture out
nEvents = 100000
q_vec = list()
scalars = list()
targets = list()
data0 = {key:[] for key in cols_to_save}
data1 = {key:[] for key in cols_to_save}
i=0
for iEvent in tqdm(range(0,nEvents)):
    if not pythia.next(): continue
    event = pythia.event
    ki  = event[2].p() #in e
    kf = event[6].p() # out e
    P  = event[1].p() # in p 
    q  = ki - kf
    Q2 = -q.m2Calc() # Q^2
    xB = Q2/(2*(P*q)) #Bjorken X
    y  = P*q/(P*ki)
    s  = (ki+P).mCalc()  
    mat = pythia8.RotBstMatrix()
    mat.toCMframe(q, P*xB*2)
    vec = P*xB+q
    vec.rotbst(mat)
    scalars.append([Q2,xB,y,s])
    targets.append(to_array(vec)) #breit target
    q_vec.append(to_array(q)) #lab q #lab P is know (100,0,0, ~99.997)
    for prt in event:
        #if selection(prt) or prt.index()in[1,2,5]:
            prt_dict = getData(prt)
            prt_dict['EVENT'] = i
            for key in cols_to_save:
                data0[key].append(prt_dict[key])
    for prt in event:
        #if selection(prt) or prt.index()in[1,2,5] or prt:
            prt.rotbst(mat)
            prt_dict = getData(prt)
            prt_dict['EVENT'] = i
            for key in cols_to_save:
                data1[key].append(prt_dict[key])
    i+=1
                
df0 = pd.DataFrame(data0)
df1 = pd.DataFrame(data1)
df0 = df0.astype(convert_dict)
df1 = df1.astype(convert_dict)
df0.to_hdf("data/EIC/lab0.h5", key=f"data",complib='zlib', complevel=9, format='table')
df1.to_hdf("data/EIC/breit0.h5", key=f"data",complib='zlib', complevel=9, format='table')
pd.DataFrame(targets,columns=['E','px', 'py','pz'], dtype='float32').to_hdf("data/EIC/targets_breit0.h5",key='data')
pd.DataFrame(q_vec,columns=['E','px', 'py','pz'], dtype='float32').to_hdf("data/EIC/q_lab0.h5",key='data')
pd.DataFrame(scalars, columns=['Q2', 'xB', 'y', 's'], dtype='float32').to_hdf("data/EIC/scalars0.h5",key='data')

100%|██████████| 100000/100000 [01:26<00:00, 1155.55it/s]


In [18]:
df = pd.read_hdf("data/EIC/breit0.h5", key=f"data")
scalars = pd.read_hdf("data/EIC/scalars0.h5", key=f"data")
targets = pd.read_hdf("data/EIC/targets_breit0.h5", key=f"data")

In [19]:
print(scalars.shape)
print(targets.shape)
print(df['EVENT'].max())

(99995, 4)
(99995, 4)
99994


In [ ]:
Rs = np.linspace(np.pi/4,3*np.pi/4,100)
losses = []
for R in Rs:
    loss = 0
    for i in range(df['EVENT'].max()+1):
        d = df[(df['EVENT']==i)&(df['STATUS']>0)][1:]
        psi = (np.pi - d.theta.values)
        Jet_region = d[R < psi.round(10)]
        pred = Jet_region.loc[:,['E','px','py','pz']].sum().values
        target = targets.iloc[i]
        loss += np.mean((pred-target)**2)
    loss /= df['EVENT'].max()
    losses.append(loss)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,5),dpi=120)
plt.plot(Rs,losses, label= "Cone")
plt.axhline(.6,ls="--",c='red',label="EFN")
plt.legend()
#plt.yscale("log")
plt.ylabel("quadratic error")
plt.xlabel("R")
plt.show()